In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import tensorflow as tf
import warnings
import tensorflow_hub as hub
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [ ]:
test_articles = pd.read_csv('../dataset/test_articles_newyork.csv', delimiter=',')

In [ ]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_newyork_max_recall5.csv', delimiter=',')

In [ ]:
embedding_model = Doc2Vec.load('../pre_trained_models/doc2vec.bin')

In [ ]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [ ]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [ ]:
test_articles.info()

In [ ]:
fixed_index.info()

In [ ]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_key_match']
        article_title_text = str(row['article_title'])
        article_meta_description_text = str(row['article_meta_description'])

        #embedding and model variables
        article_title = []
        title_table = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_key_match']]
        
        vector_words = tknzr.tokenize(article_title_text+" "+article_meta_description_text)
        word_embedding = embedding_model.infer_vector(vector_words)
        article_title.append(word_embedding)

        #creating embedding 
        for i, row in return_index.iterrows():
            
            vector_words = tknzr.tokenize(str(row['table_page_title'])+" "+str(row['table_page_main_passage']))
            word_embedding = embedding_model.infer_vector(vector_words)
            title_table.append(word_embedding)
        
        distance_vector = pairwise_distances(article_title, title_table, metric='cosine')
        
        #creating the final dataframe
        for i in range(0,len(distance_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],distance_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")